In [ ]:
from google.colab import drive
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Yêu cầu quyền truy cập vào Google Drive
drive.mount('/content/drive', force_remount=True)

# Đường dẫn thư mục project
project_dir = '/content/drive/My Drive/Project II/'

# Kiểm tra xem project_dir có tồn tại không trước khi thay đổi thư mục làm việc
if os.path.exists(project_dir):
    os.chdir(project_dir)
    print("Nội dung trong thư mục:")
    print(os.listdir())  # Liệt kê nội dung thư mục
else:
    print(f"Thư mục không tồn tại: {project_dir}")

Mounted at /content/drive
Nội dung trong thư mục:
['results', 'preprocess', 'data', 'models', 'other']


# **1. Khai báo thư viện**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import Tensor

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.modules.loss import _Loss

import scipy.sparse as sp
import random
from scipy.special import expit
from tqdm import tqdm
from collections import defaultdict
from sklearn.preprocessing import MinMaxScaler

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# === NeuMF ===
model_dir = '/content/drive/My Drive/Project II/models/ckpt/neumf_full_corpus.pth'

# **2. Load và chia dữ liệu**

## **2.1. Chia dữ liệu theo chiến lược Full-Corpus**

In [ ]:
# Load dữ liệu
data_dir = project_dir + "data/"
dataset = pd.read_csv(data_dir + "recommendations_processed.csv").copy()  # Thêm .copy() để tạo bản sao rõ ràng

# Explicitly convert 'date' column to datetime objects
dataset['date'] = pd.to_datetime(dataset['date'])

# Loại trừ duplicate
dataset = dataset.sort_values("date")  # Sort theo thời gian
dataset = dataset.drop_duplicates(subset=['user_id', 'app_id'], keep='last')

# Mapping user_id, game_id sang user_index, game_index
all_user_ids = dataset['user_id'].unique()
all_game_ids = dataset['app_id'].unique()

user_id_mapping = {user_id: idx for idx, user_id in enumerate(sorted(all_user_ids))}
game_id_mapping = {item_id: idx for idx, item_id in enumerate(sorted(all_game_ids))}

# Cập nhật lại user_id, app_id sử dụng .loc
dataset.loc[:, 'user_id'] = dataset['user_id'].map(user_id_mapping)
dataset.loc[:, 'app_id'] = dataset['app_id'].map(game_id_mapping)

In [ ]:
def split_data_full_corpus(df, ratio=0.918):
    # Với tỉ lệ thời gian 0.918 thì sẽ ra được tỉ lệ số lượng ~80%
    df = df.sort_values("date")
    start_date = df["date"].min()
    end_date = df["date"].max()
    pivot_date = start_date + (end_date - start_date) * ratio
    train_set = df[df["date"] < pivot_date].copy()
    test_set = df[df["date"] >= pivot_date].copy()
    return train_set, test_set

# Chia dữ liệu theo chiến lược full corpus
train_df, test_df = split_data_full_corpus(dataset)
train_df, valid_df = split_data_full_corpus(train_df)

print(f"train_df: {train_df.shape}")
print(f"valid_df: {valid_df.shape}")
print(f"test_df: {test_df.shape}")

train_df: (1571114, 8)
valid_df: (486039, 8)
test_df: (477880, 8)


## **2.2. Thống kê đơn giản**

In [ ]:
# Số lượng users, games
num_users = len(all_user_ids)
num_games = len(all_game_ids)
num_nodes = num_users + num_games

print(f"Số lượng users: {num_users}")
print(f"Số lượng games: {num_games}")
print(f"Số lượng nodes: {num_nodes}")

# Print cold-start statistics
train_users = set(train_df['user_id'].unique())
train_items = set(train_df['app_id'].unique())

valid_users = set(valid_df['user_id'].unique())
valid_items = set(valid_df['app_id'].unique())

test_users = set(test_df['user_id'].unique())
test_items = set(test_df['app_id'].unique())

valid_cold_start_users = valid_users - train_users
valid_cold_start_items = valid_items - train_items
test_cold_start_users = test_users - train_users
test_cold_start_items = test_items - train_items

print("Full-Corpus Evaluation")
print("\n=== Cold-start analysis in valid ===")
print(f"Total users in valid: {len(valid_users)}")
print(f"--> Cold-start users in valid: {len(valid_cold_start_users)}")
print(f"Total items in valid: {len(valid_items)}")
print(f"--> Cold-start games in valid: {len(valid_cold_start_items)}")

print("\n=== Cold-start analysis in test ===")
print(f"Total users in test: {len(test_users)}")
print(f"--> Cold-start users in test: {len(test_cold_start_users)}")
print(f"Total items in test: {len(test_items)}")
print(f"--> Cold-start games in test: {len(test_cold_start_items)}")

Số lượng users: 47274
Số lượng games: 4632
Số lượng nodes: 51906
Full-Corpus Evaluation

=== Cold-start analysis in valid ===
Total users in valid: 45039
--> Cold-start users in valid: 903
Total items in valid: 4121
--> Cold-start games in valid: 524

=== Cold-start analysis in test ===
Total users in test: 47274
--> Cold-start users in test: 903
Total items in test: 4609
--> Cold-start games in test: 1029


# **3. Các hàm tiện ích**

## **3.1. Các lớp bao dữ liệu**

In [ ]:
class Data:
    def __init__(self, pos_users, pos_items, all_neg_items, user_to_idx):
        self.pos_users = pos_users
        self.pos_items = pos_items
        self.all_neg_items = all_neg_items
        self.user_to_idx = user_to_idx

class FullCorpusData:
    def __init__(self, labels, has_interacted_masks, valid_users, warm_item_mask):
        self.labels = labels  # Ground truth labels [num_users, num_games]
        self.has_interacted_masks = has_interacted_masks  # Boolean mask of interactions [num_users, num_games]
        self.valid_users = valid_users  # Tensor of user IDs
        self.warm_item_mask = warm_item_mask  # Boolean mask of warm items [num_games]

class TestData:
    def __init__(self, labels: torch.Tensor, has_interacted_masks: torch.Tensor,
                 user_ids: torch.Tensor, warm_items_mask: torch.Tensor,
                 warm_users_mask: torch.Tensor):
        self.labels = labels  # Ground truth labels [num_users, num_games]
        self.has_interacted_masks = has_interacted_masks  # Boolean mask of interactions [num_users, num_games]
        self.user_ids = user_ids  # Tensor of user IDs
        self.warm_items_mask = warm_items_mask  # Boolean mask of warm items [num_games]
        self.warm_users_mask = warm_users_mask  # Boolean mask of warm users [num_users]

## **3.2. Hàm tạo ma trận tương tác user-item**

In [ ]:
def create_interact_matrix(dataset: pd.DataFrame, num_users: int, num_items: int) -> torch.BoolTensor:
    # Chỉ lấy các tương tác positive trên toàn bộ dataset
    pos_df = dataset[dataset['is_recommended'] == 1]
    rows = pos_df['user_id'].to_numpy()
    cols = pos_df['app_id'].to_numpy()
    data = np.ones_like(rows, dtype=np.bool_)
    mat = sp.coo_matrix((data, (rows, cols)), shape=(num_users, num_items))
    return torch.from_numpy(mat.toarray())  # dtype=bool mặc định

full_matrix = create_interact_matrix(dataset, num_users, num_games)
train_matrix = create_interact_matrix(train_df, num_users, num_games)
valid_matrix = create_interact_matrix(valid_df, num_users, num_games)

## **3.3. Hàm định nghĩa các Ranking Metrics**

In [ ]:
def precision_at_k(scores: torch.Tensor, labels: torch.Tensor, k: int) -> torch.Tensor:
    topk = torch.topk(scores, k, dim=1).indices
    hits = torch.gather(labels, 1, topk)
    return hits.sum(dim=1) / k

def recall_at_k(scores: torch.Tensor, labels: torch.Tensor, k: int) -> torch.Tensor:
    topk = torch.topk(scores, k, dim=1).indices
    hits = torch.gather(labels, 1, topk)
    relevant = labels.sum(dim=1).clamp(min=1e-8)  # avoid divide by zero
    return hits.sum(dim=1) / relevant

def ndcg_at_k(scores: torch.Tensor, labels: torch.Tensor, k: int) -> torch.Tensor:
    device = scores.device
    topk = torch.topk(scores, k, dim=1).indices
    hits = torch.gather(labels, 1, topk)

    weights = torch.log2(torch.arange(2, k + 2, device=device).float())
    dcg = (hits / weights).sum(dim=1)

    ideal_len = labels.sum(dim=1).clamp(max=k).long()
    idcg = torch.stack([
        (1.0 / weights[:L]).sum() if L > 0 else torch.tensor(0.0, device=device)
        for L in ideal_len
    ])
    return dcg / idcg.clamp(min=1e-8)

def hitrate_at_k(scores: torch.Tensor, labels: torch.Tensor, k: int) -> torch.Tensor:
    topk = torch.topk(scores, k, dim=1).indices
    hits = torch.gather(labels, 1, topk)
    return (hits.sum(dim=1) > 0).float()

## **3.4. Hàm mất mát BPR**

In [ ]:
class BPRLoss(_Loss):
    def __init__(self, lambda_reg: float = 1e-3):
        super().__init__()
        self.lambda_reg = lambda_reg

    def forward(self,
                pos_score: Tensor,
                neg_score: Tensor,
                parameters: Tensor = None) -> Tensor:
        """
        pos_score: [batch_size]
        neg_score: [batch_size, num_neg]
        parameters: embedding cần regularize
        """
        # Ensure pos_score has shape [batch_size, 1] for broadcasting
        if pos_score.dim() == 1:
            pos_score = pos_score.unsqueeze(1)  # [batch_size, 1]

        log_prob = F.logsigmoid(pos_score - neg_score).mean()

        regularization = 0
        if self.lambda_reg != 0 and parameters is not None:
            regularization = self.lambda_reg * parameters.norm(p=2).pow(2)
            regularization = regularization / pos_score.size(0)

        return -log_prob + regularization

# **4. Định nghĩa mô hình NeuMF**

In [ ]:
class NeuMF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, layers, dropout):
        super(NeuMF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items

        # GMF embeddings
        self.user_emb_gmf = nn.Embedding(num_users, embedding_dim)
        self.item_emb_gmf = nn.Embedding(num_items, embedding_dim)

        # MLP embeddings
        self.user_emb_mlp = nn.Embedding(num_users, layers[0] // 2)
        self.item_emb_mlp = nn.Embedding(num_items, layers[0] // 2)

        # MLP layers
        mlp = []
        for i in range(len(layers) - 1):
            mlp.append(nn.Linear(layers[i], layers[i+1]))
            mlp.append(nn.ReLU())
            if dropout:
                mlp.append(nn.Dropout(dropout))
        self.mlp = nn.Sequential(*mlp)

        # Prediction layer
        self.head = nn.Linear(layers[-1] + embedding_dim, 1)

        self._init_weights()

    def _init_weights(self):
        # Initialize embeddings
        nn.init.normal_(self.user_emb_gmf.weight, std=0.01)
        nn.init.normal_(self.item_emb_gmf.weight, std=0.01)
        nn.init.normal_(self.user_emb_mlp.weight, std=0.01)
        nn.init.normal_(self.item_emb_mlp.weight, std=0.01)

        # Initialize MLP layers
        for layer in self.mlp:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)

        # Initialize prediction layer
        nn.init.xavier_uniform_(self.head.weight)
        nn.init.zeros_(self.head.bias)

    def forward(self, user_idx, item_idx):
        # GMF path
        user_emb_gmf = self.user_emb_gmf(user_idx)  # [batch_size, embedding_dim]
        item_emb_gmf = self.item_emb_gmf(item_idx)  # [batch_size, embedding_dim]
        gmf_vector = user_emb_gmf * item_emb_gmf    # [batch_size, embedding_dim]

        # MLP path
        user_emb_mlp = self.user_emb_mlp(user_idx)  # [batch_size, layers[0]//2]
        item_emb_mlp = self.item_emb_mlp(item_idx)  # [batch_size, layers[0]//2]
        mlp_vector = torch.cat([user_emb_mlp, item_emb_mlp], dim=1)  # [batch_size, layers[0]]
        mlp_vector = self.mlp(mlp_vector)  # [batch_size, layers[-1]]

        # Combine paths
        vector = torch.cat([gmf_vector, mlp_vector], dim=1)  # [batch_size, layers[-1] + embedding_dim]
        scores = self.head(vector).squeeze(-1)  # [batch_size]
        return scores

    def compute_bpr_loss(self, pos_scores: torch.Tensor,
                         neg_scores: torch.Tensor,
                         lambda_reg: float = 1e-3) -> torch.Tensor:
        loss_fn = BPRLoss(lambda_reg=lambda_reg)

        all_params = []
        for param in self.parameters():
            if param.requires_grad:
                all_params.append(param.view(-1))

        concatenated_params = torch.cat(all_params) if all_params else None
        return loss_fn(pos_scores, neg_scores, concatenated_params)

# **5. Huấn luyện mô hình**

## **5.1. Định nghĩa hàm train**

In [ ]:
def train(model, train_data, train_loader, scheduler, optimizer, device, lambda_reg=1e-3, criterion='bpr', num_neg=3):
    model.train()
    total_loss = 0.0
    total_examples = 0

    num_users = model.num_users
    num_items = model.num_items

    pos_users = train_data.pos_users.to(device)
    pos_items = train_data.pos_items.to(device)
    all_neg_items = train_data.all_neg_items.to(device)
    user_to_train_idx = train_data.user_to_idx

    for batch_index in tqdm(train_loader, desc="Training", leave=False):
        # === Positive Samples ===
        batch_pos_users = pos_users[batch_index]
        batch_pos_items = pos_items[batch_index]
        batch_size = batch_pos_users.size(0)

        # === Negative Sampling ===
        batch_train_indices = torch.tensor(
            [user_to_train_idx[u.item()] for u in batch_pos_users],
            dtype=torch.long, device=device
        )
        user_neg_items = all_neg_items[batch_train_indices]  # [batch_size, num_items]
        neg_items = torch.multinomial(user_neg_items.float(), num_samples=num_neg, replacement=True)  # [batch_size, num_neg]

        # Expand users to match negative samples
        batch_neg_users = batch_pos_users.unsqueeze(1).expand(-1, num_neg).reshape(-1)  # [batch_size * num_neg]
        batch_neg_items = neg_items.reshape(-1)  # [batch_size * num_neg]

        # === Compute scores ===
        pos_scores = model(batch_pos_users, batch_pos_items)  # [batch_size]  # Use batch_pos_items instead of batch_neg_items
        neg_scores = model(batch_neg_users, batch_neg_items)  # [batch_size * num_neg]
        neg_scores = neg_scores.view(batch_size, num_neg)  # [batch_size, num_neg]

        optimizer.zero_grad()
        if criterion == 'bpr':
            loss = model.compute_bpr_loss(pos_scores=pos_scores,
                                        neg_scores=neg_scores,
                                        lambda_reg=lambda_reg)
        else:
            raise ValueError(f"Unknown Loss Type: {criterion}")

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch_size
        total_examples += batch_size

    scheduler.step()
    train_loss = total_loss / total_examples if total_examples > 0 else 0.0
    return train_loss

## **5.2. Định nghĩa hàm validate**

In [ ]:
@torch.no_grad()
def validate(model, valid_data, valid_loader, device, criterion='bpr', num_neg=3):
    model.eval()
    total_loss = 0.0
    total_examples = 0

    num_users = model.num_users
    num_items = model.num_items

    pos_users = valid_data.pos_users.to(device)
    pos_items = valid_data.pos_items.to(device)
    all_neg_items = valid_data.all_neg_items.to(device)
    user_to_valid_idx = valid_data.user_to_idx

    for batch_index in tqdm(valid_loader, desc="Validating", leave=False):
        # === Positive Samples ===
        batch_pos_users = pos_users[batch_index]
        batch_pos_items = pos_items[batch_index]
        batch_size = batch_pos_users.size(0)

        # === Negative Sampling ===
        batch_valid_indices = torch.tensor(
            [user_to_valid_idx[u.item()] for u in batch_pos_users],
            dtype=torch.long, device=device
        )
        user_neg_items = all_neg_items[batch_valid_indices]  # [batch_size, num_items]
        neg_items = torch.multinomial(user_neg_items.float(), num_samples=num_neg, replacement=True)  # [batch_size, num_neg]

        # Expand users to match negative samples
        batch_neg_users = batch_pos_users.unsqueeze(1).expand(-1, num_neg).reshape(-1)  # [batch_size * num_neg]
        batch_neg_items = neg_items.reshape(-1)  # [batch_size * num_neg]

        # === Compute scores ===
        pos_scores = model(batch_pos_users, batch_pos_items)  # [batch_size]  # Use batch_pos_items instead of batch_neg_items
        neg_scores = model(batch_neg_users, batch_neg_items)  # [batch_size * num_neg]
        neg_scores = neg_scores.view(batch_size, num_neg)  # [batch_size, num_neg]

        # === Compute loss ===
        loss = model.compute_bpr_loss(pos_scores=pos_scores,
                                    neg_scores=neg_scores,
                                    lambda_reg=0.0)

        total_loss += loss.item() * batch_size
        total_examples += batch_size

    valid_loss = total_loss / total_examples if total_examples > 0 else 0.0
    return valid_loss

## **5.3. Định nghĩa hàm evaluate**

In [ ]:
@torch.no_grad()
def evaluate_full_corpus(model, full_corpus_data, device, k=10, batch_size=512, item_batch_size=500):
    model.eval()

    valid_labels = full_corpus_data.labels.to(device)
    has_interacted_masks = full_corpus_data.has_interacted_masks.to(device)
    users_global = full_corpus_data.valid_users.to(device)
    warm_item_mask = full_corpus_data.warm_item_mask.to(device)

    # Filter for warm-start items
    valid_labels = valid_labels[:, warm_item_mask]
    has_interacted_masks = has_interacted_masks[:, warm_item_mask]

    all_ndcg = []
    all_hit = []

    num_valid = users_global.size(0)
    num_items = warm_item_mask.sum().item()  # Use number of warm items instead of all items

    for start in tqdm(range(0, num_valid, batch_size), desc="Evaluating Full-Corpus", leave=False):
        end = min(start + batch_size, num_valid)
        batch_users = users_global[start:end]

        # Initialize scores tensor for this batch of users
        batch_scores = torch.full((batch_users.size(0), num_items), -float('inf'), device=device)

        # Process items in chunks
        for item_start in range(0, num_items, item_batch_size):
            item_end = min(item_start + item_batch_size, num_items)

            # Create user-item pairs for this chunk
            batch_users_expanded = batch_users.unsqueeze(1).expand(-1, item_end - item_start)
            chunk_items = torch.arange(item_start, item_end, device=device).unsqueeze(0).expand(batch_users.size(0), -1)

            # Reshape for model forward pass
            batch_users_flat = batch_users_expanded.reshape(-1)
            chunk_items_flat = chunk_items.reshape(-1)

            # Get scores for this chunk
            chunk_scores = model(batch_users_flat, chunk_items_flat)

            # Reshape scores back to original dimensions
            chunk_scores = chunk_scores.reshape(batch_users.size(0), item_end - item_start)

            # Store scores in the appropriate slice
            batch_scores[:, item_start:item_end] = chunk_scores

            # Clear memory
            del chunk_scores
            torch.cuda.empty_cache()

        # Mask out items user has already interacted with
        batch_scores = batch_scores.masked_fill(has_interacted_masks[start:end], -float('inf'))

        # Get labels for this batch
        labels = valid_labels[start:end]

        # Calculate metrics
        ndcg = ndcg_at_k(batch_scores, labels, k)
        hitrate = hitrate_at_k(batch_scores, labels, k)

        all_ndcg.append(ndcg)
        all_hit.append(hitrate)

        # Clear memory
        del batch_scores
        torch.cuda.empty_cache()

    avg_ndcg = torch.cat(all_ndcg).mean().item() if all_ndcg else 0.0
    avg_hitrate = torch.cat(all_hit).mean().item() if all_hit else 0.0
    return avg_ndcg, avg_hitrate

## **5.4. Train Loss, Valid Loss, NDCG@10, HitRate@10**

### **5.3.1. Chuẩn bị dữ liệu trước khi huấn luyện**

In [ ]:
# Create mapping from global user indices to training user indices
train_users_tensor = torch.tensor(list(train_users), dtype=torch.long)
train_num_users = train_users_tensor.size(0)
user_to_train_idx = {u.item(): i for i, u in enumerate(train_users_tensor)}

# Extract positive interactions from train_df
train_pos_df = train_df[train_df['is_recommended'] == 1]
train_pos_users = torch.tensor(train_pos_df['user_id'].values, dtype=torch.long)
train_pos_items = torch.tensor(train_pos_df['app_id'].values, dtype=torch.long)

# Pre-compute all possible negative items for each user in train dataset
train_all_neg_items = torch.zeros((train_num_users, num_games), dtype=torch.bool)
for i, u in enumerate(train_users_tensor):
    train_all_neg_items[i] = ~full_matrix[u]

# Create Data object
train_data = Data(
    pos_users=train_pos_users,
    pos_items=train_pos_items,
    all_neg_items=train_all_neg_items,
    user_to_idx=user_to_train_idx
)

In [ ]:
valid_users_in_matrix = torch.tensor(list(valid_users), dtype=torch.long)
valid_num_users = valid_users_in_matrix.size(0)
user_to_valid_idx = {u.item(): i for i, u in enumerate(valid_users_in_matrix)}

# Extract positive interactions from valid_df
valid_pos_df = valid_df[valid_df['is_recommended'] == 1]
valid_pos_users = torch.tensor(valid_pos_df['user_id'].values, dtype=torch.long)
valid_pos_items = torch.tensor(valid_pos_df['app_id'].values, dtype=torch.long)

# Pre-compute negative items for validation dataset
valid_all_neg_items = torch.zeros((valid_num_users, num_games), dtype=torch.bool)
# Iterate through the mapped user IDs present in the validation matrix
for i, mapped_user_id in enumerate(valid_users_in_matrix):
    # Use the mapped_user_id to index the full_matrix (or valid_matrix if preferred, but full_matrix is consistent with train)
    valid_all_neg_items[i] = ~full_matrix[mapped_user_id]

# Create Data object
valid_data = Data(
    pos_users=valid_pos_users,
    pos_items=valid_pos_items,
    all_neg_items=valid_all_neg_items,
    user_to_idx=user_to_valid_idx
)

In [ ]:
# Create data loader
train_loader = DataLoader(
    dataset = range(len(train_data.pos_users)),
    batch_size = 8192,
    shuffle = True,
    num_workers = 2,
    pin_memory=True
)

# Create data loader
valid_loader = DataLoader(
    dataset=range(len(valid_data.pos_users)),
    batch_size=8192,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

In [ ]:
def create_full_corpus_data(dataset_df, train_df, valid_df, num_games, device, min_neg=9):
    # Build game_list from the entire dataset
    game_list = sorted(dataset_df["app_id"].unique())
    game_to_idx = {game_id: idx for idx, game_id in enumerate(game_list)}
    num_games_filtered = len(game_list)  # Should now be equal to num_games

    # Danh sách user trong validation
    valid_user_ids = sorted(set(valid_df["user_id"]))
    user_to_idx = {user_id: idx for idx, user_id in enumerate(valid_user_ids)}
    num_valid_users = len(valid_user_ids)

    # Tạo cấu trúc dữ liệu cho user_played_games (từ train set)
    user_played_games = defaultdict(set)
    for user_id, game_id in zip(train_df["user_id"], train_df["app_id"]):
        user_played_games[user_id].add(game_id)

    # Tạo cấu trúc dữ liệu cho user_valid (từ valid set)
    user_valid = defaultdict(list)
    for user_id, game_id, label in zip(valid_df["user_id"], valid_df["app_id"], valid_df["is_recommended"]):
        user_valid[user_id].append((game_id, label))

    # Khởi tạo valid_labels và has_interacted_masks
    valid_labels = torch.zeros((num_valid_users, num_games_filtered), device = device, dtype=torch.int)
    has_interacted_masks = torch.zeros((num_valid_users, num_games_filtered), device = device, dtype=torch.bool)

    # Gán nhãn từ valid
    for user_id, interactions in user_valid.items():
        if user_id not in user_to_idx:
            continue
        u_idx = user_to_idx[user_id]
        for game_id, label in interactions:
            if game_id in game_to_idx:
                g_idx = game_to_idx[game_id]
                valid_labels[u_idx, g_idx] = label

    # Gán mask từ train
    for user_id, played_games in user_played_games.items():
        if user_id not in user_to_idx:
            continue
        u_idx = user_to_idx[user_id]
        for game_id in played_games:
            if game_id in game_to_idx:
                g_idx = game_to_idx[game_id]
                has_interacted_masks[u_idx, g_idx] = True

    # Lọc user có đủ số tương tác âm (min_neg)
    good_users = []
    for user_id in valid_user_ids:
        played = user_played_games.get(user_id, set())
        positives = {g for g, l in user_valid[user_id] if l == 1}
        unplayed = set(game_list) - played - positives
        if len(unplayed) >= min_neg:
            good_users.append(user_id)

    # Cập nhật danh sách và tensors chỉ với good_users
    keep_indices = [user_to_idx[u] for u in good_users]
    valid_labels = valid_labels[keep_indices]
    has_interacted_masks = has_interacted_masks[keep_indices]
    valid_users_tensor = torch.tensor(good_users, device = device, dtype=torch.long)

    # Mask cho warm items (game vừa có trong train vừa có trong valid)
    train_items = set(train_df["app_id"])
    valid_items = set(valid_df["app_id"])
    warm_items = train_items & valid_items
    warm_items_mask = torch.tensor([g in warm_items for g in game_list], device = device, dtype=torch.bool)

    return FullCorpusData(
        labels = valid_labels,
        has_interacted_masks = has_interacted_masks,
        valid_users = valid_users_tensor,
        warm_item_mask = warm_items_mask
    )

In [ ]:
# Call the function with the full dataset and num_games
full_corpus_data = create_full_corpus_data(dataset, train_df, valid_df, num_games, device)

### **5.3.2. Khởi tạo mô hình và huấn luyện**

In [ ]:
model = NeuMF(
    num_users = num_users,
    num_items = num_games,
    embedding_dim = 64,
    layers = [128, 64, 32],
    dropout = 0.4
).to(device)

print("Tham số của mô hình:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"param_name: {name}, param_size: {param.size()}, requires_grad: {param.requires_grad}")

Tham số của mô hình:
param_name: user_emb_gmf.weight, param_size: torch.Size([47274, 8]), requires_grad: True
param_name: item_emb_gmf.weight, param_size: torch.Size([4632, 8]), requires_grad: True
param_name: user_emb_mlp.weight, param_size: torch.Size([47274, 16]), requires_grad: True
param_name: item_emb_mlp.weight, param_size: torch.Size([4632, 16]), requires_grad: True
param_name: mlp.0.weight, param_size: torch.Size([16, 32]), requires_grad: True
param_name: mlp.0.bias, param_size: torch.Size([16]), requires_grad: True
param_name: mlp.3.weight, param_size: torch.Size([8, 16]), requires_grad: True
param_name: mlp.3.bias, param_size: torch.Size([8]), requires_grad: True
param_name: head.weight, param_size: torch.Size([1, 16]), requires_grad: True
param_name: head.bias, param_size: torch.Size([1]), requires_grad: True


In [ ]:
optimizer = optim.Adam(model.parameters(), lr = 0.01)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 50, eta_min = 0.0005)
num_epochs = 50

train_losses, valid_losses, ndcgs_10, hitrates_10 = [], [], [], []
best_ndcg_10 = 0.0
best_hitrate_10 = 0.0

for epoch in range(1, num_epochs + 1):
   train_loss = train(
       model = model,
       train_data = train_data,
       train_loader = train_loader,
       optimizer = optimizer,
       scheduler = scheduler,
       device = device,
       lambda_reg = 0.001,
       criterion = 'bpr',
       num_neg = 3
   )

   valid_loss = validate(
       model = model,
       valid_data = valid_data,
       valid_loader = valid_loader,
       device = device,
       criterion = 'bpr',
       num_neg = 3
   )

   ndcg_10, hitrate_10 = evaluate_full_corpus(model, full_corpus_data, device, k = 10)

   train_losses.append(train_loss)
   valid_losses.append(valid_loss)
   ndcgs_10.append(ndcg_10)
   hitrates_10.append(hitrate_10)

   print(f"Epoch: {epoch:02d} | Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f} | NDCG@10: {ndcg_10:.4f} | HitRate@10: {hitrate_10:.4f}")

   if ndcg_10 > best_ndcg_10 and hitrate_10 > best_hitrate_10:
      best_ndcg_10 = ndcg_10
      best_hitrate_10 = hitrate_10
      if os.path.exists(model_dir):
          os.remove(model_dir)

      torch.save(model.state_dict(), model_dir)
      print("---> Best checkpoint is saved!")

Epoch: 01 | Train Loss: 0.4576 | Valid Loss: 0.8603 | NDCG@10: 0.0089 | HitRate@10: 0.0804
---> Best checkpoint is saved!


Epoch: 02 | Train Loss: 0.3032 | Valid Loss: 0.8716 | NDCG@10: 0.0094 | HitRate@10: 0.0668


Epoch: 03 | Train Loss: 0.2549 | Valid Loss: 0.9025 | NDCG@10: 0.0082 | HitRate@10: 0.0565


Epoch: 04 | Train Loss: 0.2368 | Valid Loss: 0.9037 | NDCG@10: 0.0078 | HitRate@10: 0.0551


Epoch: 05 | Train Loss: 0.2283 | Valid Loss: 0.9538 | NDCG@10: 0.0080 | HitRate@10: 0.0545


...

Epoch: 09 | Train Loss: 0.2146 | Valid Loss: 1.0275 | NDCG@10: 0.0066 | HitRate@10: 0.0488


Epoch: 10 | Train Loss: 0.2126 | Valid Loss: 1.0782 | NDCG@10: 0.0067 | HitRate@10: 0.0481


Epoch: 11 | Train Loss: 0.2106 | Valid Loss: 1.0810 | NDCG@10: 0.0070 | HitRate@10: 0.0482


Epoch: 12 | Train Loss: 0.2094 | Valid Loss: 1.1094 | NDCG@10: 0.0076 | HitRate@10: 0.0498


Training:  46%|████▌     | 72/157 [00:11<00:11,  7.46it/s]

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))  # 1 hàng, 3 cột

# Create a range of epoch numbers for the x-axis
epochs_range = range(1, len(train_losses) + 1)

# --- Plot 1: Training and Validation Losses ---
axes[0].plot(epochs_range, train_losses, label='Train Loss')
axes[0].plot(epochs_range, valid_losses, label='Valid Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Train and Validation Loss')
axes[0].legend()
axes[0].grid(True)

# --- Plot 2: NDCG@10 over Epochs ---
axes[1].plot(epochs_range, ndcgs_10, label='NDCG@10')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Metric Value')
axes[1].set_title('NDCG@10 over Epochs')
axes[1].legend()
axes[1].grid(True)

# --- Plot 3: HitRate@10 over Epochs ---
axes[2].plot(epochs_range, hitrates_10, label='HitRate@10')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Metric Value')
axes[2].set_title('HitRate@10 over Epochs')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.show()

# **6. Đánh giá khả năng xếp hạng (ranking)**

In [ ]:
model = NeuMF(
    num_users = num_users,
    num_items = num_games,
    embedding_dim = 64,
    layers = [128, 64, 32],
    dropout = 0.2
).to(device)

# Load model with proper device mapping
model.load_state_dict(torch.load(model_dir, map_location=device))
model.to(device)  # Move model to appropriate device

In [ ]:
def create_test_data(dataset_df, train_df, test_df, device, min_unplayed=9):
    # Create game list and mappings
    game_list = sorted(dataset_df["app_id"].unique())
    game2idx = {game_id: idx for idx, game_id in enumerate(game_list)}
    num_games_filtered = len(game_list)

    # Identify warm-start and cold-start users
    train_user_ids = set(train_df["user_id"])
    test_user_ids = set(test_df["user_id"])
    warm_user_ids = sorted(train_user_ids & test_user_ids)
    cold_user_ids = sorted(test_user_ids - train_user_ids)
    all_test_user_ids = warm_user_ids + cold_user_ids
    user2idx = {user_id: idx for idx, user_id in enumerate(all_test_user_ids)}

    # Track interacted games for warm-start users (train)
    user_played_games = defaultdict(set)
    for user_id, game_id in zip(train_df["user_id"], train_df["app_id"]):
        if user_id in user2idx:
            user_played_games[user_id].add(game_id)

    # Track test interactions for all users
    user_test = defaultdict(list)
    for user_id, game_id, label in zip(test_df["user_id"], test_df["app_id"], test_df["is_recommended"]):
        user_test[user_id].append((game_id, label))

    # Initialize tensors
    num_test_users = len(all_test_user_ids)
    test_labels = torch.zeros((num_test_users, num_games_filtered), dtype=torch.float, device=device)
    has_interacted_masks = torch.zeros((num_test_users, num_games_filtered), dtype=torch.bool, device=device)

    # Fill tensors
    for user_id in all_test_user_ids:
        u_idx = user2idx[user_id]
        for game_id, label in user_test[user_id]:
            if game_id in game2idx:
                test_labels[u_idx, game2idx[game_id]] = float(label)
        for game_id in user_played_games.get(user_id, []):
            if game_id in game2idx:
                has_interacted_masks[u_idx, game2idx[game_id]] = True

    # Filter users with too few unplayed games
    good_users = []
    for user_id in all_test_user_ids:
        played = user_played_games.get(user_id, set())
        positives = {g for g, l in user_test[user_id] if l == 1}
        unplayed = set(game_list) - played - positives
        if len(unplayed) >= min_unplayed:
            good_users.append(user_id)

    keep_indices = [user2idx[u] for u in good_users]
    test_labels = test_labels[keep_indices]
    has_interacted_masks = has_interacted_masks[keep_indices]
    warm_users_mask = torch.tensor([u in train_user_ids for u in good_users],
                                   dtype=torch.bool, device=device)
    user_ids_tensor = torch.tensor(good_users, dtype=torch.long, device=device)

    # Create mask for warm-start items
    train_items = set(train_df["app_id"])
    warm_items_mask = torch.tensor([g in train_items for g in game_list],
                                   dtype=torch.bool, device=device)

    return TestData(
        labels=test_labels,
        has_interacted_masks=has_interacted_masks,
        user_ids=user_ids_tensor,
        warm_items_mask=warm_items_mask,
        warm_users_mask=warm_users_mask
    )

test_data = create_test_data(dataset, train_df, test_df, device)

In [ ]:
@torch.no_grad()
def evaluate_ranking_full_corpus(model, test_data, device, k=10, batch_size=512, item_batch_size=500):
    print("Evaluating ranking metrics on full corpus...")
    model.eval()

    # Get test data
    test_labels = test_data.labels.to(device)
    has_interacted_masks = test_data.has_interacted_masks.to(device)
    user_ids = test_data.user_ids.to(device)
    warm_items_mask = test_data.warm_items_mask.to(device)
    warm_users_mask = test_data.warm_users_mask.to(device)

    # Metrics storage
    all_prec = []
    all_recall = []
    all_ndcg = []
    all_hit = []

    num_test = user_ids.size(0)
    num_items = model.num_items

    # Process users in batches
    for start in tqdm(range(0, num_test, batch_size), desc="Evaluating", leave=False):
        end = min(start + batch_size, num_test)
        batch_idx = slice(start, end)
        batch_user_ids = user_ids[batch_idx]
        batch_warm_mask = warm_users_mask[batch_idx]

        # Initialize scores for all users with negative infinity
        batch_scores = torch.full((end-start, num_items), -float('inf'), device=device)

        # Only process warm-start users
        if batch_warm_mask.any():
            warm_user_indices = torch.where(batch_warm_mask)[0]
            warm_user_ids = batch_user_ids[warm_user_indices]

            # Process items in chunks for each warm user
            for item_start in range(0, num_items, item_batch_size):
                item_end = min(item_start + item_batch_size, num_items)

                # Create user-item pairs for this chunk
                batch_users_expanded = warm_user_ids.unsqueeze(1).expand(-1, item_end - item_start)
                chunk_items = torch.arange(item_start, item_end, device=device).unsqueeze(0).expand(warm_user_ids.size(0), -1)

                # Reshape for model forward pass
                batch_users_flat = batch_users_expanded.reshape(-1)
                chunk_items_flat = chunk_items.reshape(-1)

                # Get scores for this chunk
                chunk_scores = model(batch_users_flat, chunk_items_flat)

                # Reshape scores back to original dimensions
                chunk_scores = chunk_scores.reshape(warm_user_ids.size(0), item_end - item_start)

                # Store scores in the appropriate positions
                for i, orig_idx in enumerate(warm_user_indices):
                    batch_scores[orig_idx, item_start:item_end] = chunk_scores[i]

                # Clear memory
                del chunk_scores
                torch.cuda.empty_cache()

        # Mask scores for already interacted items
        batch_scores = batch_scores.masked_fill(has_interacted_masks[batch_idx], -float('inf'))
        batch_labels = test_labels[batch_idx]

        # Skip users with no positive labels
        valid_users = batch_labels.sum(dim=1) > 0
        if not valid_users.any():
            continue

        batch_scores = batch_scores[valid_users]
        batch_labels = batch_labels[valid_users]

        # Compute metrics
        prec = precision_at_k(batch_scores, batch_labels, k)
        recall = recall_at_k(batch_scores, batch_labels, k)
        ndcg = ndcg_at_k(batch_scores, batch_labels, k)
        hitrate = hitrate_at_k(batch_scores, batch_labels, k)

        all_prec.append(prec)
        all_recall.append(recall)
        all_ndcg.append(ndcg)
        all_hit.append(hitrate)

        # Clear memory
        del batch_scores
        torch.cuda.empty_cache()

    # Compute average metrics
    all_prec = torch.cat(all_prec).mean().item() if all_prec else 0.0
    all_recall = torch.cat(all_recall).mean().item() if all_recall else 0.0
    all_ndcg = torch.cat(all_ndcg).mean().item() if all_ndcg else 0.0
    all_hit = torch.cat(all_hit).mean().item() if all_hit else 0.0

    return {
        f"Precision@{k}": all_prec,
        f"Recall@{k}": all_recall,
        f"NDCG@{k}": all_ndcg,
        f"HitRate@{k}": all_hit,
    }